In [13]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input, RepeatVector, TimeDistributed, Dropout


In [14]:
# Create output dir
os.makedirs("models", exist_ok=True)

In [15]:
# Load data with faults
sensor_data = pd.read_csv("sensor_data.csv", parse_dates=['timestamp'])
sensor_data.drop(columns=['timestamp'], inplace=True)


In [26]:
sensor_data.shape

(1440, 6)

In [16]:
# Separate features and labels
features = sensor_data.drop(columns=['fault','Machine_name']).values
labels = sensor_data['fault'].values


In [17]:
# Normalize features
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

In [18]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.2, random_state=42)

(1152, 4)

In [19]:
# ====================
# Row-Based Deep Autoencoder
# ====================
input_dim = X_train.shape[1]
input_layer = Input(shape=(input_dim,))
encoded = Dense(16, activation='relu')(input_layer)
encoded = Dropout(0.1)(encoded)
encoded = Dense(8, activation='relu')(encoded)
decoded = Dense(16, activation='relu')(encoded)
decoded = Dropout(0.1)(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

In [20]:
model = Model(input_layer, decoded)
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, X_train, epochs=50, batch_size=32, validation_split=0.1)

Epoch 1/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0418 - val_loss: 0.0439
Epoch 2/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0401 - val_loss: 0.0425
Epoch 3/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0380 - val_loss: 0.0398
Epoch 4/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0362 - val_loss: 0.0350
Epoch 5/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0315 - val_loss: 0.0293
Epoch 6/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0271 - val_loss: 0.0240
Epoch 7/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0215 - val_loss: 0.0197
Epoch 8/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0200 - val_loss: 0.0167
Epoch 9/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0194 - val_loss: 0.0149
Epoch 10/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0166 - val_loss: 0.0138
Epoch 11/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0150 - val_loss: 0.0127
Epoch 12/50
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0157 - val_lo

In [21]:
# Save model
model.save("models/deep_autoencoder_anomaly.h5")

In [22]:
# Predict anomalies
X_pred = model.predict(X_test)
mse = np.mean(np.power(X_test - X_pred, 2), axis=1)
threshold = np.percentile(mse, 95)  # 95th percentile threshold
y_pred = [1 if e > threshold else 0 for e in mse]

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


In [23]:
# Save predictions
predictions_df = pd.DataFrame({'mse': mse, 'anomaly': y_pred})
predictions_df.to_csv("models/row_autoencoder_predictions.csv", index=False)

In [24]:
predictions_df 

,mse,anomaly
0,0.007003,0
1,0.000517,0
2,0.000648,0
3,0.001254,0
4,0.008132,0
...,...,...
283,0.001908,0
284,0.001474,0
285,0.001461,0
286,0.001660,0
